In [1]:
import pandas as pd
import numpy as np # Usaremos para definir os limites do gradiente da terceira tabela

In [2]:
# --- Configurações ---
arquivo_topics = 'topics.xlsx'
arquivo_systems = 'systems.xlsx'

In [3]:
# --- Carregamento dos Dados ---
try:
    df_topics = pd.read_excel(arquivo_topics, index_col=0) # A primeira coluna é o índice
    df_systems = pd.read_excel(arquivo_systems, index_col=0) # A primeira coluna é o índice
except FileNotFoundError:
    print(f"Erro: Um ou ambos os arquivos Excel ('{arquivo_topics}', '{arquivo_systems}') não foram encontrados.")
    print("Por favor, crie arquivos de exemplo com os dados fornecidos ou substitua pelos seus arquivos.")
    # Criando dados de exemplo se os arquivos não existirem, para que o script possa rodar
    data_topics = {
        'Topic': ['Abstract', 'Airplane', 'Aliens', 'Alternate History', 'Assassin'],
        'Action': [2, 3, 3, 3, 3],
        'Adventure': [3, -3, 1, 1, -2],
        'RPG': [-3, 1, 3, 3, 3],
        'Simulation': [-3, 3, -3, 1, 1],
        'Strategy': [1, 3, 2, 2, -3],
        'Casual': [-3, 3, -2, -3, -3],
        'Y': [1, 3, 2, -3, -3],
        'E': [2, 3, 3, 3, 1],
        'M': [3, 2, 3, 3, 3]
    }
    df_topics = pd.DataFrame(data_topics).set_index('Topic')
    df_topics.to_excel(arquivo_topics)
    print(f"Arquivo '{arquivo_topics}' de exemplo criado.")

    data_systems = {
        'Topic': ['PC', 'G64', 'TES', 'Master V', 'Gameling', 'Vena Gear'], # 'Topic' aqui é o nome do sistema
        'Action': [2, 2, 1, 2, 1, 2],
        'Adventure': [3, 3, -2, -2, -2, 1],
        'RPG': [2, 2, 1, 1, 2, 1],
        'Simulation': [3, 2, 1, 1, 2, 2],
        'Strategy': [3, 3, -2, -2, -3, -3],
        'Casual': [-3, -2, 3, 3, 3, 3],
        'Y': [1, 1, 3, 2, 3, 2],
        'E': [2, 2, -2, 3, 2, 3],
        'M': [3, 3, 1, -2, -3, 1]
    }
    df_systems = pd.DataFrame(data_systems).set_index('Topic') # Mantendo 'Topic' como nome do índice por consistência com o exemplo
    df_systems.to_excel(arquivo_systems)
    print(f"Arquivo '{arquivo_systems}' de exemplo criado.")
    print("\n--- Rodando com dados de exemplo ---")

In [4]:
# --- Função para Aplicar Gradiente de Cores ---
# Para as tabelas originais, o range é de -3 a 3
def aplicar_estilo_original(df):
    return df.style.background_gradient(cmap='RdYlGn', vmin=-3, vmax=3).format("{:.0f}")

In [5]:
# Para a tabela cruzada, o range será diferente.
# Cada célula da tabela cruzada é a soma de uma célula de df_topics e uma de df_systems.
# Portanto, o valor mínimo possível é -3 + (-3) = -6, e o máximo é 3 + 3 = 6.
# Se tivermos N colunas de atributos, o score total para uma combinação Topic x System
# será a soma desses valores combinados.
# min_score_combinado_total = N * (-6)
# max_score_combinado_total = N * (6)
def aplicar_estilo_cruzado(df, num_atributos):
    min_val = num_atributos * (-3 + -3) # Pior cenário: todos os atributos são -3 para topic e -3 para system
    max_val = num_atributos * (3 + 3)   # Melhor cenário: todos os atributos são 3 para topic e 3 para system
    return df.style.background_gradient(cmap='RdYlGn', vmin=min_val, vmax=max_val).format("{:.0f}")

In [6]:
# --- 1. Exibir Tabela de Topics com Gradiente ---
print("\n--- Tabela de Topics ---")
styled_topics = aplicar_estilo_original(df_topics)
display(styled_topics)


--- Tabela de Topics ---


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
Topic,,,,,,,,,
Abstract,2,3,-3,-3,1,-3,1,2,3
Airplane,3,-3,1,3,3,3,3,3,2
Aliens,3,1,3,-3,2,-2,2,3,3
Alternate History,3,1,3,1,2,-3,-3,3,3
Assassin,3,-2,3,1,-3,-3,-3,1,3
Business,-3,1,1,3,3,-3,2,3,-2
City,-2,-3,-2,3,3,-2,2,3,1
Colonization,-2,-3,-3,3,3,-2,-2,3,1
Comedy,-3,3,1,-3,-3,3,1,2,3


In [7]:
# --- 2. Exibir Tabela de Systems com Gradiente ---
print("\n--- Tabela de Systems ---")
# Renomear o índice para "System" para clareza na tabela cruzada
df_systems_renomeado = df_systems.rename_axis("System")
styled_systems = aplicar_estilo_original(df_systems_renomeado)
display(styled_systems)


--- Tabela de Systems ---


,Action,Adventure,RPG,Simulation,Strategy,Casual,Y,E,M
System,,,,,,,,,
PC,2,3,2,3,3,-3,1,2,3
G64,2,3,2,2,3,-2,1,2,3
TES,1,-2,1,1,-2,3,3,-2,1
Master V,2,-2,1,1,-2,3,2,3,-2
Gameling,1,-2,2,2,-3,3,3,2,-3
Vena Gear,2,1,1,2,-3,3,2,3,1
Vena Oasis,3,1,1,2,-3,-2,1,3,2
Super TES,2,2,2,3,-2,2,3,2,-2
Playsystem,3,1,3,2,-2,-3,1,3,2


In [8]:
# --- 3. Criar e Exibir Tabela de Cruzamento de Dados ---
print("\n--- Tabela de Cruzamento (Melhores Combinações Topic x System) ---")

# Verificar se as colunas de atributos são as mesmas (exceto o índice)
# Isso é importante para a soma fazer sentido
common_attributes = df_topics.columns.intersection(df_systems.columns)
if len(common_attributes) != len(df_topics.columns) or len(common_attributes) != len(df_systems.columns):
    print("Aviso: As colunas de atributos não são idênticas entre topics e systems.")
    print(f"Usando apenas colunas comuns: {list(common_attributes)}")
    df_topics_filtered = df_topics[common_attributes]
    df_systems_filtered = df_systems[common_attributes]
else:
    df_topics_filtered = df_topics
    df_systems_filtered = df_systems

# Inicializar DataFrame para os resultados do cruzamento
# Linhas serão os Topics, Colunas serão os Systems
df_crossed = pd.DataFrame(index=df_topics_filtered.index, columns=df_systems_renomeado.index, dtype=float)

# Calcular o score de combinação para cada par (Topic, System)
for topic_name, topic_data in df_topics_filtered.iterrows():
    for system_name, system_data in df_systems_renomeado.iterrows(): # Usar df_systems_renomeado para ter o nome do índice correto
        # Somar os scores dos atributos correspondentes
        combined_scores = topic_data[common_attributes] + system_data[common_attributes] # system_data também deve ser filtrado por common_attributes
        # O score total para esta combinação é a soma desses scores combinados
        df_crossed.loc[topic_name, system_name] = combined_scores.sum()

# Aplicar estilo à tabela cruzada
# O número de atributos é o número de colunas usadas na soma
num_atributos_usados = len(common_attributes)
styled_crossed = aplicar_estilo_cruzado(df_crossed, num_atributos_usados)
display(styled_crossed)

print("\n--- Explicação da Tabela de Cruzamento ---")
print(f"Cada célula na tabela de cruzamento representa a soma das pontuações de um 'Topic' e um 'System'")
print(f"para todos os {num_atributos_usados} atributos (gêneros e faixas etárias).")
print(f" - Valores mais altos (verdes) indicam melhores combinações.")
print(f" - Valores mais baixos (vermelhos) indicam piores combinações.")
print(f"Os limites de cor para esta tabela são ajustados: de {num_atributos_usados * -6} a {num_atributos_usados * 6}.")


--- Tabela de Cruzamento (Melhores Combinações Topic x System) ---


System,PC,G64,TES,Master V,Gameling,Vena Gear,Vena Oasis,Super TES,Playsystem,TES 64,DreamVast,Playsystem 2,mBox,Game Sphere,GS,PPS,mBox 360,Nuu,Playsystem 3,grPhone,grPad,mPad,Wuu,OYA,mBox One,Playsystem 4,mBox Next,Playsystem 5,Custom Console
Topic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abstract,19,19,7,9,8,15,11,15,13,12,8,18,13,10,23,16,19,7,18,8,9,7,11,16,13,18,17,15,3
Airplane,34,34,22,24,23,30,26,30,28,27,23,33,28,25,38,31,34,22,33,23,24,22,26,31,28,33,32,30,18
Aliens,28,28,16,18,17,24,20,24,22,21,17,27,22,19,32,25,28,16,27,17,18,16,20,25,22,27,26,24,12
Alternate History,26,26,14,16,15,22,18,22,20,19,15,25,20,17,30,23,26,14,25,15,16,14,18,23,20,25,24,22,10
Assassin,16,16,4,6,5,12,8,12,10,9,5,15,10,7,20,13,16,4,15,5,6,4,8,13,10,15,14,12,0
Business,21,21,9,11,10,17,13,17,15,14,10,20,15,12,25,18,21,9,20,10,11,9,13,18,15,20,19,17,5
City,19,19,7,9,8,15,11,15,13,12,8,18,13,10,23,16,19,7,18,8,9,7,11,16,13,18,17,15,3
Colonization,14,14,2,4,3,10,6,10,8,7,3,13,8,5,18,11,14,2,13,3,4,2,6,11,8,13,12,10,-2
Comedy,20,20,8,10,9,16,12,16,14,13,9,19,14,11,24,17,20,8,19,9,10,8,12,17,14,19,18,16,4



--- Explicação da Tabela de Cruzamento ---
Cada célula na tabela de cruzamento representa a soma das pontuações de um 'Topic' e um 'System'
para todos os 9 atributos (gêneros e faixas etárias).
 - Valores mais altos (verdes) indicam melhores combinações.
 - Valores mais baixos (vermelhos) indicam piores combinações.
Os limites de cor para esta tabela são ajustados: de -54 a 54.
